In [ ]:
import torch
from collections import defaultdict
import math, os, sys, time, json, pickle, tarfile
from os import path
from PIL import Image
from os.path import expanduser as expand, join
import mat4py
if '..' not in sys.path:
    sys.path.append('..')
import ulfs

att_filepath = expand('~/data/imagenet/attrann.mat')
img_data = expand('~/data/imagenet/att_synsets')

mat = mat4py.loadmat(att_filepath)
print('mat[attrann.keys()', mat['attrann'].keys())
print('mat[attrann[images][:10]', mat['attrann']['images'][:10])
print('len(mat[attrann[images])', len(mat['attrann']['images']))
print('mat[attrann[attributes]', mat['attrann']['attributes'])
print('len(mat[attrann[attributes])', len(mat['attrann']['labels']))
print('len(mat[attrann[attributes][0])', len(mat['attrann']['labels'][0]))
att_t = torch.LongTensor(mat['attrann']['labels'])
att_t = att_t.clamp(min=0)
print('att_t.size()', att_t.size())

image_keys = set()
images_by_key = defaultdict(set)
for image in mat['attrann']['images']:
    split_image = image[0].split('_')
    image_keys.add(split_image[0])
    images_by_key[split_image[0]].add(split_image[1])

def print_untar_cmd():
    untar_str = 'tar -xf /efs-hp-coldstore/data/fall11_whole.tar'
    for image_key in sorted(list(image_keys)):
        untar_str += ' ' + image_key + '.tar'
    print(untar_str)

# print_untar_cmd()

has = set()
has_not = set()
for image_key in sorted(list(image_keys)):
    if path.isfile(join(img_data, f'{image_key}.tar')):
        has.add(image_key)
    else:
        has_not.add(image_key)
print('has', len(has), 'not', len(has_not))

def select_and_shrink_images():
    if not path.isdir(join(img_data, 'att_images')):
        os.makedirs(join(img_data, 'att_images'))
    for image_key, images in sorted(images_by_key.items()):
        print(image_key)
        tar = tarfile.open(join(img_data, f'{image_key}.tar'))
        names = tar.getnames()
        member_keys = [name.split('.')[0].split('_')[1] for name in names]
        for im in images:
            f = tar.extractfile(f'{image_key}_{im}.JPEG')
            i = Image.open(f)
            i = i.resize([32, 32], resample=Image.BICUBIC)
            i.save(join(img_data, 'att_images', f'{image_key}_{im}.JPEG'))

# select_and_shrink_images()

In [ ]:
import time
import torch
import numpy as np
from PIL import Image

att_images = [i[0] for i in mat['attrann']['images']]
print('att_images[:5]', att_images[:5])

N = len(att_images)
images_t = torch.Tensor(N, 3, 32, 32)
last_print = time.time()
for n in range(N):
    i = Image.open(join(img_data, 'att_images', att_images[n] + '.JPEG'))
    i = torch.from_numpy(np.asarray(i))
    i = i.float() / 255.0
    i = i.transpose(1, 2).transpose(0, 1)
    images_t[n] = i
    if time.time() - last_print >= 5.0:
        print(n)
        last_print = time.time()

print('saving...')
with open(join(img_data, 'att_images_t.dat'), 'wb') as f:
    torch.save(images_t, f)
print('... saved')